1. Import Libraries and dependencies

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import mediapipe as mp

2. KeyPoints using MP Holistic

In [ ]:
# mp_holistic = mp.solutions.holistic
# mp_drawing = mp.solutions.drawing_utils

In [ ]:
# def mediapipe_detection(image, model):
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image.flags.writeable = False                  
#     results = model.process(image)                 
#     image.flags.writeable = True                   
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
#     return image, results

In [ ]:
# def draw_landmarks(image, results):
#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
#     mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
#     mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 

In [ ]:
# def draw_styled_landmarks(image, results):
    
#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
#                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
#                              mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
#                              ) 
    
#     mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
#                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
#                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
#                              ) 
      
#     mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
#                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
#                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
#                              ) 

In [ ]:
# cap = cv2.VideoCapture(0)

# with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.8) as holistic:
#     while cap.isOpened():

        
#         ret, frame = cap.read()

       
#         image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        
#         draw_styled_landmarks(image, results)

       
#         cv2.imshow('OpenCV Feed', image)

        
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

3. Extract KeyPoints

In [ ]:
# def extract_keypoints(results):
#     pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
#     lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
#     rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
#     return np.concatenate([pose, lh, rh])

4. SetUp Folders for Data Collection

In [ ]:
DATA_PATH = os.path.join('MP_Data') 

actions = np.array(['Indian', 'Sign', 'Language', 'Man', 'Woman', 'Namaste', 'Deaf', 'Sorry', 'Happy', 'Sad', 'Understand', 'Bye', 'Please', 'Food', 'Water'])

no_sequences = 30

sequence_length = 30

start_folder = 30

In [ ]:
# for action in actions: 
#     for sequence in range(no_sequences):
#         try: 
#             os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
#         except:
#             pass

5. Collect KeyPoints for training and testing

In [ ]:
# cap = cv2.VideoCapture(0)
 
# with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.8) as holistic:
    
   
#     for action in actions:
      
#         for sequence in range(no_sequences):
           
#             for frame_num in range(sequence_length):

                
#                 ret, frame = cap.read()

                
#                 image, results = mediapipe_detection(frame, holistic)

#                 draw_styled_landmarks(image, results)
                
               
#                 if frame_num == 0: 
#                     cv2.putText(image, 'STARTING COLLECTION', (120,100), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,153, 0), 3, cv2.LINE_AA)
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (14,22), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 153), 2, cv2.LINE_AA)
                    
#                     cv2.imshow('OpenCV Feed', image)
#                     cv2.waitKey(300)
#                 else: 
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (14,22), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 153), 2, cv2.LINE_AA)
                 
#                     cv2.imshow('OpenCV Feed', image)
                
               
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)

#                 if cv2.waitKey(10) & 0xFF == ord('q'):
#                     break
                    
#     cap.release()
#     cv2.destroyAllWindows()

6. Preprocess Data and Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

In [ ]:
X = np.array(sequences)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=1000)

In [ ]:
y_test.shape

In [ ]:
X_train.shape

7. Build and Train LSTM Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=250, callbacks=[tb_callback], validation_data = (X_test, y_test))

In [ ]:
model.summary()

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('MODEL ACCURACY')
plt.ylabel('ACCURACY')
plt.xlabel('EPOCH')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('MODEL LOSS')
plt.ylabel('LOSS')
plt.xlabel('EPOCH')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

8. Make Predictions

In [ ]:
yhat = model.predict(X_test)

In [ ]:
# actions[np.argmax(yhat[4])]

In [ ]:
# actions[np.argmax(y_test[4])]

9. Save Weights

In [ ]:
# model.save('actions,88%-test_size=0.1,random_state=1000.h5')

10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

In [ ]:
ytrue = np.argmax(y_test, axis=1)
yhat = np.argmax(yhat, axis=1)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
cf_matrix = confusion_matrix(ytrue, yhat,labels=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])
cm = ConfusionMatrixDisplay(confusion_matrix=cf_matrix, display_labels=actions)
cm.plot(xticks_rotation='vertical')

In [ ]:
cr = classification_report(ytrue, yhat)
print(cr)

11. Test in Real Time

In [ ]:
# accuracy=[]
# sequence = []
# sentence = []
# predictions = []
# threshold = 0.9

# cap = cv2.VideoCapture(0)

# with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.8) as holistic:
#     while cap.isOpened():

     
#         ret, frame = cap.read()

       
#         image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
       
#         draw_styled_landmarks(image, results)
        
        
#         keypoints = extract_keypoints(results)
#         sequence.append(keypoints)
#         sequence = sequence[-30:]
        
#         if len(sequence) == 30:
#             res = model.predict(np.expand_dims(sequence, axis=0))[0]
#             # print(actions[np.argmax(res)])
#             predictions.append(np.argmax(res))
            
        
#             if np.unique(predictions[-10:])[0]==np.argmax(res): 
#                 if res[np.argmax(res)] > threshold:
                    
#                     if len(sentence) > 0: 
#                         if actions[np.argmax(res)] != sentence[-1]:
#                             # print(actions[np.argmax(res)])
#                             sentence.append(actions[np.argmax(res)])
#                             accuracy.append(str(round(int(res[np.argmax(res)]*100),2)))
#                     else:
#                         # print(actions[np.argmax(res)])
#                         sentence.append(actions[np.argmax(res)])
#                         accuracy.append(str(round(int(res[np.argmax(res)]*100),2)))

#         if len(sentence) > 1: 
#             sentence = sentence[-1:]
#             accuracy = accuracy[-1:]
        
#         string = str(sentence);  
#         count = 0;  
        
#         for i in range(0, len(string)):  
#             if(string[i] != ' '):  
#                 count = count + 1
        
#         if (count==7):
#             count = count + 1
#         elif (count==14):
#             count = count - 2   

            
#         cv2.rectangle(image, (0,0), (count*30, 40), (245, 117, 16), -1)
#         cv2.putText(image,''.join(sentence)+' - '+''.join(accuracy)+'%', (15,30), 
#                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
       
#         cv2.imshow('OpenCV Feed', image)

        
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()